In [16]:
import time
from multiprocessing import Pool
from requests import Session
from requests.adapters import HTTPAdapter
from typing import List
import  requests

In [18]:
class DppApi(object):

    def __init__(self, username, password,s=Session()):
        # requests = RequestApi()
        self.url = 'http://dpp.backend.patsnap.com'
        self.username = username
        self.password = password
        self.sess = s
        self.adapter = HTTPAdapter(max_retries=10)
        self.sess.mount('http://', self.adapter)
        self.sess.mount('https://', self.adapter)
        self.headers =self.api_headers()





    def get(self, url, **kwargs):
        return self.sess.get(url, **kwargs)

    def post(self, url, data=None, json=None, **kwargs):
        return self.sess.post(url, data, json, **kwargs)

    # @retry(stop_max_attempt_number=20)
    def get_token(self):
        url = self.url + '/api/auth/login'
        headers = {'X-Requested-With': 'XMLHttpRequest'}
        body = {
            'username': self.username,
            'password': self.password
        }
        res = self.sess.post(url=url, json=body, headers=headers).json()
        return res['token']

    def api_headers(self):
        return {'Content-Type': 'application/json', 'X-Requested-With': 'XMLHttpRequest',
                'X-Authorization': 'Bearer ' + self.get_token()}

    def get_task_details(self,
                         name: str = None,
                         task_type: int = 1,
                         s_own_task: bool = False,
                         search_tags: list = [],
                         status: str = '',
                         taskId: int = None,
                         time: List[str] = '',
                         size: int = 10,
                         current: int = 1
                         ):
        """

        @param name: 任务名称
        @param task_type: 任务类型
                           1 PATENT          14 DATA_LAKE
                           2 TRADEMARK       15 TERCHBLOG
                           5 NPL             16 CUSTOM
                           6 NEWS            18 TECH_OFFERING
                           7 LICENSE         19 CHEMICAL
                           8 LEGAL           20 BIO
                           9 TRANSFRE        21 FUNDED_RESEARCH
                           10 COMPANY        22 BIG_DATA
                           11 MARKETREPORT   23 INNOVATION
                           12 GRANT
                           13 APPEAL
        @param s_own_task: 只看自己
        @param search_tags: 标签
        @param status: 状态
                            0 CREATING      5 PREPARING        990 ARCHINING
                            1 PENDING       6 PREPARED         991 ARCHIVED
                            2 RUNNING       7 STARTING         992 RECOVERING
                            3 DONE          100 STATISTICING   993 RECOVERED
                            4 CREATED       400 KILLED         -1 FAILED
        @param taskId: 任务ID
        @param size: 自定义返回数量
        @param current: 当前页数
        @param time: 时间范围 example: ["2021-07-31T16:00:00.000Z","2021-08-05T16:00:00.000Z"]
        @return:
        """
        url = self.url + '/api/task/list'
        body = {
            "query": {
                "size": size,
                "current": current,
                "task_type": task_type,
                "name": name,
                "is_own_task": s_own_task,
                "search_tags": search_tags,
                "filters_map": {
                    "status": status,
                    "taskId": taskId,
                    "time": time
                }
            }
        }
        res = requests.post(url=url, json=body, headers=self.headers).json()
        return res

    # 数据api详情
    def get_patent_detail(self, key, table='patent'):
        """

        @param key: 公开号 or  patent_id,是个list，e.g.  ['JP6676120B2', 'JP6678534B2', 'JP6677357B2']
        @param table: original:官网原始的数据
                      patent:线上展示的数据, patent = original + docdb合并后并标准化后的线上数据
                      docdb:另一个第三方数据源
        """
        url = self.url + '/api/tools/searchPatent'
        body = {
            'itemKeys': key,
            'type': table
        }
        res = requests.post(url=url, json=body, headers=self.headers).json()
        return res

    # 数据api 源文件下载地址
    def get_patent_file(self, pn_list, table='patent'):
        url = self.url + '/api/tools/listS3Files'
        body = {
            'itemKeys': pn_list,
            'type': table
        }
        res = requests.post(url=url, json=body, headers=self.headers).json()
        return res

    # 获取页面展示的99条数据
    def get_part_of_db_data(self, logId):
        db_result_url = self.url + f'/api/athena/getresult?logId={logId}'
        print(db_result_url)
        response =self.sess.get(url=db_result_url, headers=self.headers)
        db_res = response.json()
        res = db_res['result']['rows']
        while res != 'SUCCEEDED':
            time.sleep(1)
            response = self.sess.get(url=db_result_url, headers=self.headers)
            res = response.json()['result']['state']
        return response.json()['result']['rows']

    # 获取数据库文件下载地址（所有数据）
    def get_all_db_data(self, logId):
        url = 'http://dpp.backend.patsnap.com/api/athena/retrygetdownloadurl'
        body = {"athenaLogId": logId}
        response = requests.post(url=url, json=body, headers=self.headers)
        download_url = response.json()['result']
        return download_url

    def query_athena(self, sql, database='patent_dw', account='747875099153'):
        """

        @param sql: sql语句
        @param database: 数据库
        @param account: aws用户名0066 or 7478 or 3977
        @return:
        """
        print(self.sess)
        url = self.url + '/api/athena/query'
        body = {
            "sql": sql,
            "database": database,
            "athenaAccount": account
        }
        res = requests.post(url=url, json=body, headers=self.headers).json()
        logId = res['result']['logId']
        print(logId)
        return self.get_part_of_db_data(logId), self.get_all_db_data(logId)

    def query_athena_multiprocessing(self, sql_list, threading_num):
        """

        @param sql_list: 要查询的sql列表
        @param threading_num: 每次请求发送的数量，即进程数，建议填写cpu核心数
        @return: list，每条sql对应的查询结果，默认取第一个返回结果，即最多100条记录
        """
        pool = Pool(threading_num)
        # 用lamba切割成 {{threading_num}} 个一组的list, [[...], [...], [...], [...], ...]
        fun = lambda a: map(lambda b: a[b:b + threading_num], range(0, len(a), threading_num))
        result = []
        for _data in list(fun(sql_list)):

            ret = pool.map(self.query_athena, _data)
            for i in ret:
                result.append(i[0])
        return result


In [43]:
dpp = DppApi(username='caiqijun@patsnap.com', password='caiqijun1234')

In [58]:
dw_patent=dpp.query_athena('select * from finance_data.v2_20211117sse_company_patent_indicators')

919805
http://dpp.backend.patsnap.com/api/athena/getresult?logId=919805


In [59]:
dw_patent=dw_patent[0]
dw_patent=pd.DataFrame(dw_patent)

In [60]:
dw_patent=dw_patent[['name','ans_id','id','country_cnt','patenting_growth_ratio','avg_remaining_life_span','top5_current_quality_index',
                   'in_examing_ratio','external_licensing_cnt','granted_invention_cnt','non_design_apno_cnt','apno_cnt','avg_3y_cnt',
                   'joint_applicant_cnt','value_sum','avg_technology_score','ip_dependency','avg_cited_by_cnt','tech_width','value_avg',
                   'pct_apno_cnt','active_inventor_ratio','value_top5_avg','invention_stability','core_patents_cited_by_ratio',
                   'core_patents_cited_by_cnt','tech_global_concentration','most_cited_patents_value','total_patent_award_score',
                    'granted_invention_ratio','self_cited_by_ratio','top5_current_impact_index','invention_ratio','joint_application_cnt']]

In [61]:
dw_patent

,name,ans_id,id,country_cnt,patenting_growth_ratio,avg_remaining_life_span,top5_current_quality_index,in_examing_ratio,external_licensing_cnt,granted_invention_cnt,...,core_patents_cited_by_ratio,core_patents_cited_by_cnt,tech_global_concentration,most_cited_patents_value,total_patent_award_score,granted_invention_ratio,self_cited_by_ratio,top5_current_impact_index,invention_ratio,joint_application_cnt
0,华邦古楼新材料有限公司,ad2c4d55-0557-46cc-b86f-89a79cd0aa65,275567,1,0.9364916731037085,16.258082191780822,0.16,0.4186046511627907,0,11,...,11.0,22,2.2129641211164885E-4,161600.0,0,0.255813953488372093,0.11627906976744186,1.4761826275868999,0.813953488372093,17
1,浙江宇轻科技有限公司,164c2775-5d54-4322-a697-a38cec92903f,779501,1,NaN,8.747945205479454,0.0,0.0,0,0,...,NaN,NaN,1.8739997526320326E-5,NaN,0,0.000000000000000000,0.0,0.0,0.0,NaN
2,浙江汇胜信息科技有限公司,44563b4e-3c3c-4866-a250-b9f8a85a3577,626115,1,NaN,8.936986301369863,0.0,0.0,0,0,...,NaN,NaN,1.327948631822336E-5,NaN,0,0.000000000000000000,0.0,0.0,0.0,NaN
3,青昂钢管（上海）有限公司,80b1943d-4150-461c-ac3f-dfc8debdee63,424823,1,0.0,9.27945205479452,0.0,0.11764705882352941,0,0,...,NaN,NaN,1.3235601650744238E-5,NaN,0,0.000000000000000000,0.0,0.0,0.11764705882352941,NaN
4,三原禛食汇食品有限公司,e390614c-c97e-4840-b07c-4804545f4576,94866,1,-0.5,7.06027397260274,0.0,0.0,0,0,...,NaN,NaN,0.0,NaN,0,0.000000000000000000,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,厦门市鑫雅锋针织机械有限公司,f4f8d45d-1264-48b0-8e54-878acded0e25,36597,1,-0.5,5.616438356164384,0.0,0.0,0,0,...,NaN,NaN,2.801826791067776E-5,NaN,0,0.000000000000000000,0.0,0.0,0.0,NaN
95,沈阳巨德齿轮有限公司,e9e37a58-189b-45d0-87d5-b15d883b70cc,73820,1,1.0,NaN,0.04,0.0,0,0,...,NaN,NaN,2.2948763061670976E-5,NaN,0,0.000000000000000000,0.0,0.0,0.0,NaN
96,广州天太智能科技有限公司,d4888785-8053-4354-b3d2-58e54e9e6c66,144729,1,1.0,NaN,0.0,0.0,0,0,...,NaN,NaN,2.8802755847679507E-6,NaN,0,0.000000000000000000,0.0,0.9355969055453638,0.0,NaN
97,中石化中原石油工程有限公司,10f11978-0d9b-4d3d-8951-f4132772710a,797328,2,4.490251100164468,7.625373134328363,0.8,0.26044226044226043,0,19,...,9.9,99,2.964954461197552E-4,827600.0,0,0.046683046683046683,0.019656019656019656,2.1326385790631,0.31695331695331697,273
